In [11]:
import cupy as cp
xp = cp
n_ant = 10**5  # number of antennas
ant_1_array, ant_2_array = cp.tril_indices(n_ant, k=-1)

#only keep around number of baselines used in calibration (length set by ant indices)
n_bl = 2*len(ant_1_array)
n_gains = 2*n_ant #re im split
n_eig = 3         #number of eigenmodes
n_src = 5         #number of sources
xp = cp  

# Initialize sparse matrix Δ and edges (just for illustration)
m, n = 1000, 5  # large m, small n
Δ = cp.random.rand(m, n)  # Sparse matrix Δ
# edges = cp.array([0, 200, 400, 600, 800, 1000])  # Define block edges
edges = (xp.unique(xp.random.randint(1, int(n_bl / 2) - 1, size=(n_ant,))* 2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))


# Initialize a vector to multiply with
vec = cp.random.rand(n)

# Initialize result array
result = cp.zeros(m)

# Use multiple streams for parallel block multiplication
streams = [cp.cuda.Stream() for _ in range(len(edges) - 1)]  # One stream per block

# Create a kernel to perform block multiplication
def block_multiply(start_idx, end_idx, block, vec, result):
    result[start_idx:end_idx] = cp.matmul(block, vec)

# Launch each block multiplication in parallel
for i, stream in enumerate(streams):
    start_idx, end_idx = edges[i], edges[i + 1]
    block = Δ[start_idx:end_idx, :]  # Extract the block
    with stream:
        block_multiply(start_idx, end_idx, block, vec, result)

# Synchronize all streams to ensure completion
for stream in streams:
    stream.synchronize()

# The result now contains the multiplication result


OutOfMemoryError: Out of memory allocating 160,000,000,000 bytes (allocated so far: 10,000,057,856 bytes).

In [10]:
print("Result of block multiplication:", result)

Result of block multiplication: [0.33038739 0.67126841 0.66069762 0.86565893 1.00121492 0.51265662
 0.69001431 0.79246567 0.82701952 1.08435558 1.05037519 0.8878582
 0.5760756  0.59817478 0.84749552 0.48116514 0.85190001 0.41056955
 0.79701642 0.81742059 0.89538866 0.68838798 1.15571637 0.85544857
 0.69593025 0.65715979 0.88705789 0.77598233 0.63568065 0.70068158
 0.85328849 0.38922022 0.46164519 0.65355158 0.90719566 1.01169533
 0.65496789 0.34914356 0.68336731 0.48763377 0.5457807  0.63956327
 0.3606624  0.87837318 0.74558869 0.65952741 0.92874304 0.49210586
 0.6141663  0.66212883 1.01588478 0.75384264 0.34960611 1.14895449
 0.81403782 1.06743168 0.74006251 0.51404289 0.62538526 0.69519966
 0.51706988 0.59762643 0.4338479  0.78135274 0.38029584 0.48110485
 0.70614689 0.62750832 0.5426772  0.82808283 0.59697624 0.67470005
 0.76531767 1.09063558 1.0480132  0.89980722 0.58393855 0.53226499
 0.65305873 0.7524954  0.72719826 1.12625695 0.58588545 0.93233404
 0.50990557 0.23737895 0.846714

In [ ]:
from cupy import RawKernel

# Define the kernel code for block multiplication (assuming matrix multiply on small blocks)
kernel_code = """
extern "C" __global__
void block_multiply_kernel(const float *block, const float *vec, float *result, 
                            const int *start_idx, const int *end_idx, const int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < *end_idx - *start_idx) {
        int i = *start_idx + idx;
        float sum = 0.0f;
        for (int j = 0; j < n; j++) {
            sum += block[idx * n + j] * vec[j];
        }
        result[i] = sum;
    }
}
"""

num_blocks = len(edges) - 1  # Number of blocks

# Compile and launch the kernel
block_multiply = RawKernel(kernel_code, 'block_multiply_kernel')

# Allocate memory for result and indices
start_idx = cp.array(edges[:-1], dtype=cp.int32)
end_idx = cp.array(edges[1:], dtype=cp.int32)

# Launch the kernel (each block is processed by one thread)
block_multiply((num_blocks, 1), (block_size, 1), (Δ, vec, result, start_idx, end_idx, n))

# Synchronize streams if using multiple streams
cp.cuda.Stream.null.synchronize()


NameError: name 'num_blocks' is not defined